<a href="https://colab.research.google.com/github/kdpbiiitd/CSE508_Winter2023_A2_39./blob/main/IR_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import string
import nltk
import math
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import defaultdict
import numpy as np

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [30]:
df = pd.read_csv('dataset.csv')
df = df.drop('ArticleId', axis=1)

print(df.head())

                                                Text  Category
0  worldcom ex-boss launches defence lawyers defe...  business
1  german business confidence slides german busin...  business
2  bbc poll indicates economic gloom citizens in ...  business
3  lifestyle  governs mobile choice  faster  bett...      tech
4  enron bosses in $168m payout eighteen former e...  business


In [31]:
stopwords = nltk.corpus.stopwords.words('english')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = " ".join([word.lower() for word in text.split() if word.lower() not in stopwords])
    return text

In [32]:
df['Text'] = df['Text'].apply(clean_text)
df['Text'] = df['Text'].apply(word_tokenize)
df['Text'] = df['Text'].apply(lambda x: [stemmer.stem(word) for word in x])
df['Text'] = df['Text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [36]:
# Calculate TF
tf = defaultdict(Counter)
for i, row in df.iterrows():
    for word in row['Text']:
        tf[row['Category']][word] += 1

# Calculate ICF
N = len(df)
icf = {}
for word in set(df['Text'].sum()):
    n_t = sum([1 for i, row in df.iterrows() if word in row['Text']])
    icf[word] = math.log(N / n_t)

# Calculate TF-ICF
tf_icf = defaultdict(dict)
for label, counter in tf.items():
    for word, count in counter.items():
        tf_icf[label][word] = count * icf[word]

In [37]:
# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# Calculate the prior probabilities of each category in the training set
prior_probs = defaultdict(float)
total_count = len(train_df)
for category, count in train_df['Category'].value_counts().items():
    prior_probs[category] = count / total_count

# Calculate the likelihood probabilities of each word in each category in the training set
tf = defaultdict(Counter)
for i, row in train_df.iterrows():
    for word in row['Text']:
        tf[row['Category']][word] += 1

N = len(train_df)
icf = defaultdict(float)
for word in set(train_df['Text'].sum()):
    n_t = sum([1 for i, row in train_df.iterrows() if word in row['Text']])
    icf[word] = math.log(N / n_t)

tf_icf = defaultdict(dict)
for label, counter in tf.items():
    for word, count in counter.items():
        tf_icf[label][word] = count * icf[word]

In [38]:
# Create a dictionary to store the likelihood probabilities
likelihood_probs = defaultdict(dict)
for label, counter in tf_icf.items():
    total_count = sum(counter.values())
    for word, count in counter.items():
        likelihood_probs[label][word] = count / total_count

In [39]:
# Use the Naive Bayes algorithm to predict the category of each text in the test set
y_true = test_df['Category'].tolist()
y_pred = []
for i, row in test_df.iterrows():
    probs = defaultdict(float)
    for label in prior_probs.keys():
        probs[label] = math.log(prior_probs[label])
        for word in row['Text']:
            if word in likelihood_probs[label]:
                probs[label] += math.log(likelihood_probs[label][word])
            else:
                probs[label] += math.log(1e-9)
    y_pred.append(max(probs, key=probs.get))

In [40]:
# Calculate performance
print('Accuracy: {}'.format(accuracy_score(y_true, y_pred)))
print('Precision: {}'.format(precision_score(y_true, y_pred, average='weighted')))
print('Recall: {}'.format(recall_score(y_true, y_pred, average='weighted')))
print('F1: {}'.format(f1_score(y_true, y_pred, average='weighted')))

Accuracy: 0.9888143176733781
Precision: 0.9890153235437588
Recall: 0.9888143176733781
F1: 0.9888031097867478
